In [263]:
# 1. import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import time

In [304]:
def get_city_population(city):
    url = ("https://en.wikipedia.org/wiki/"+city) 
    response = requests.get(url)
    time.sleep(2)   #quick solution - I just added this in case of slow internet, it should be tested for 200 code but for this, it's okay
    soup = BeautifulSoup(response.content, "html.parser")
    table_string = soup.find(text=re.compile('^Population\Z')) #Looking for any text between any tags (I used \Z for end of string because $ includes \n (end of line) too)
    parent = table_string.find_parent('tr') #data that we need is in several 'tr' elements that are siblings, and the string we found is child of the first one of those elements (that's why i am looking for it's parent)
    description = []
    population = []
    population_string = parent.get_text("|",strip='True') # First 'tr' always contains 'Population' string, I've added | as separator for easier splitting since some contain only 
                                                            #'Population', some have '[2]' as reference, some have actual number for population and it always comes as last
    description.append(population_string.split('|')[0])  #Taking first element after split will give me 'Population' string
    population_string2 = population_string.split('|')[-1] #Taking last element after split will give me either 'Population','[5]'-referencies or '234,567'-actual population
    if(population_string2.startswith('[') | (population_string2 == 'Population')): #In case it gives me 'Population' or referencies
        population.append('')             #I will just append empty string where population number should be
    else:
        population.append(population_string2) # If it gives me actual number, I append number

    siblings = parent.next_siblings   # I am looking for all siblings of the first 'tr' element we found(the one with 'Population')
    for sib in siblings:
        if(sib['class'] != ['mergedrow']):    #it is a long list but we only need the few ones after our first and they all have class='mergedrow'
            break                             #as soon as loop comes into class='mergedtoprow' it will break, as we don't need those elements
        for ch in sib.children:               #all 'tr' elements that we found have children that contain data we need 
            if(ch['class']==['infobox-label']):                        #'th' child with class= 'infobox-label' contains Description string (e.g. City, Estimate, Rural, density...)
                temp = ch.get_text(strip='True').replace('\xa0',' ')   #strip=True removes all whitespaces around text. Some unnecessary '\xa0' striings were found so I just replaced them with ''
                temp = re.sub(r'[^a-zA-Z ]','',temp)                   #replacing all charachters that are not letters or whitespace
                description.append(temp)                         
            elif(ch['class']==['infobox-data']):                      #'th' child with class= 'infobox-data' contains Population string (e.g. 12,506,238 or some explanations for Ranks or density)
                population.append(ch.get_text(strip='True').replace('\xa0',' '))
    population_table = pd.DataFrame({"Description":description,          #putting everything into table
                       "Population":population
                      })
    return population_table                                              #returning table

In [305]:
print(get_city_population('Moscow')) #be careful with cities that have more than one word(should have _ between)

  Description  Population
0  Population            
1    Estimate  12,506,468
2        Rank         1st
